In [7]:
def dump_RT_KC(dump,GRID,radius):
    global gcov_kerr, gcon_kerr, N1
    #Set outputfile parameters
    thetain = 0.
    thetaout = h.max()
    phiin = 0.
    phiout = ph.max()
    for i in range(len(r[0,:,0,0])):
        if (r[0,i,0,0]>radius or i==len(r[0,:,0,0])-1):
            N1 = i
            #print(i)
            break
    #N1=len(r[0,:,0,0])
    print(N1,r[0,N1,0,0],radius,Rin)
    Rout = r[0,N1-1,0,0]
    rhoflr = (1*10**-4)(Rout**(-3/2))
    pflr = (1*10*-6)((Rout**-5/2))
    hslope=0.0
    metric = 1
    code = 2
    dim = 3
    prec = 1
    N1_offset=0
    N1=N1-N1_offset
    #Allocate new arrays for output
    uukerr=np.copy(uu)
    bukerr=np.copy(uu)
    bs1=len(r[0,:,0,0])
    bs2=len(r[0,0,:,0])
    bs3=len(r[0,0,0,:])
    print(nb, bs1, bs2, bs3)
    #Set grid parameters alpha and beta in Kerr-Schild coordinates
    beta=np.zeros((4,nb,bs1,bs2,1),dtype=np.float64)
    vkerr=np.zeros((4,nb,bs1,bs2,bs3),dtype=np.float64)
    wkerr=np.zeros((4,nb,bs1,bs2,bs3),dtype=np.float64)
    Bkerr=np.zeros((4,nb,bs1,bs2,bs3),dtype=np.float64)
    LF=uu[0]/np.sqrt(-gcon[0,0])
    #Transform to Kerr-Schild 4-velocity
    if(0):
        set_KS()
        alpha = 1./(-gcon_kerr[0,0,0])**0.5
        beta[1:4,0] = gcon_kerr[0,1:4,0]*alpha*alpha
        uukerr[:,0]=mdot(dxdxp[:,:,0], uu[:,0])
        bukerr[:,0]=mdot(dxdxp[:,:,0], bu[:,0])
        pressure=(gam-1.0)*ug
        Bkerr[1:4,0]=alpha*uukerr[0,0]*bukerr[1:4,0]-alpha*bukerr[0,0]*uukerr[1:4,0]
        vkerr[1:4,0] = (beta[1:4,0] + uukerr[1:4,0]/uukerr[0,0])/alpha
    if(1):
        alpha = 1./(-gcon[0,0,0])**0.5
        beta[1:4,0] = gcon[0,1:4,0]*alpha*alpha
        Bkerr[1:4,0]=alpha*uu[0,0]*bu[1:4,0]-alpha*bu[0,0]*uu[1:4,0]
        vkerr[1:4,0] = (beta[1:4,0] + uu[1:4,0]/uu[0,0])/alpha
        # wkerr[1:4,0] = uu[1:4,0] + uu[0,0] * beta[1:4,0]
        # wkerr[:,0]=mdot(dxdxp[:,:,0], wkerr[:,0])
        vkerr[:,0]=mdot(dxdxp[:,:,0], vkerr[:,0])
        Bkerr[:,0]=mdot(dxdxp[:,:,0], Bkerr[:,0])
        pressure=(gam-1.0)*ug
    #Start writing to ASCII
    if(0):
        f = open(dump, 'w')
        f.write('#x^1, x^2, x^3, rho, u^0, u^1, u^2, u^3, P_gas, B^1, B^2, B^3 \n')
        f.write('%d  %d  %d  %.17g %.17g  %.17g  %.17g  %.17g  %.17g  %.17g  %.17g  %.17g  %.17g  %d  %d  %d \n' % (N1,bs2,bs3,t,a,Rin,thetain,phiin,Rout,thetaout,phiout,rhoflr,pflr,metric,code,dim))
        for zs in range(0,int(bs3/4)):
            z=int(4*zs)
            if(z%10==0):
                print(z)
            for j in range (0,bs2):
                for i in range(0,N1):
                    f.write('%.9g %.9g %.9g %.9g %.9g %.9g %.9g %.9g %.9g %.9g %.9g \n' % (r[0,i,j,z],h[0,i,j,z],ph[0,i,j,z],rho[0,i,j,z],vkerr[1,0,i,j,z],vkerr[2,0,i,j,z],vkerr[3,0,i,j,z],pressure[0,i,j,z],Bkerr[1,0,i,j,z],Bkerr[2,0,i,j,z],Bkerr[3,0,i,j,z]))
        f.close()
    #Start writing to binary
    if(GRID ==0):
        import struct
        f = open(dump, 'wb+')
        header = [N1,bs2,int(bs3)]
        head = struct.pack('i'*3, *header)
        f.write(head)
        header = [t,a,Rin,thetain,phiin,Rout,thetaout,phiout,gam,hslope]
        head = struct.pack('d'*10, *header)
        f.write(head)
        header = [metric,code,dim,prec]
        head = struct.pack('i'*4, *header)
        f.write(head)
        for z in range(0,int(bs3)):
            #if(z%10==0):
                #print(z)
            for j in range (0,bs2):
                for i in range(N1_offset,N1+N1_offset):
                    #data = [rho[0,i,j,z],vkerr[1,0,i,j,z],vkerr[2,0,i,j,z],vkerr[3,0,i,j,z],pressure[0,i,j,z],Bkerr[1,0,i,j,z],Bkerr[2,0,i,j,z],Bkerr[3,0,i,j,z]]
                    #data = [rho[0,i,j,z],LF[0,i,j,z]*vkerr[1,0,i,j,z],LF[0,i,j,z]*vkerr[2,0,i,j,z],LF[0,i,j,z]*vkerr[3,0,i,j,z],pressure[0,i,j,z],
                    #        Bkerr[1,0,i,j,z],Bkerr[2,0,i,j,z],Bkerr[3,0,i,j,z],beta_e[0,i,j,z],sigma_e[0,i,j,z]]
                    data = [rho[0,i,j,z],LF[0,i,j,z]*vkerr[1,0,i,j,z],LF[0,i,j,z]*vkerr[2,0,i,j,z],LF[0,i,j,z]*vkerr[3,0,i,j,z],pressure[0,i,j,z],
                            Bkerr[1,0,i,j,z],Bkerr[2,0,i,j,z],Bkerr[3,0,i,j,z]]
                    s = struct.pack('f'*8, *data)
                    f.write(s)
        f.close()
        print('finish writing binary ' + dump)
    if(GRID==1): #grid files
        import struct
        f = open(dump, 'wb+')
        header = [N1,bs2,int(bs3)]
        head = struct.pack('i'*3, *header)
        f.write(head)
        header = [t,a,Rin,thetain,phiin,Rout,thetaout,phiout,gam,hslope]
        head = struct.pack('d'*10, *header)
        #header = [a]
        #head = struct.pack(‘d’*1, *header)
        f.write(head)
        header = [metric,code,dim,prec]
        head = struct.pack('i'*4, *header)
        f.write(head)
        for z in range(0,int(bs3)):
            #if(z%10==0):
                #print(z)
            for j in range (0,bs2):
                for i in range(N1_offset,N1+N1_offset):
                    data = [r[0,i,j,z],h[0,i,j,z],ph[0,i,j,z]]
                    #if (pressure[0,i,j,z]<0.0):
                        #print(“%d %g %d %g %d %g” %(i,r[0,i,j,z],j,h[0,i,j,z],z,ph[0,i,j,z],pressure[0,i,j,z]))
                    s = struct.pack('f'*3, *data)
                    f.write(s)
        f.close()
        print('finish writing binary ' + dump)

In [ ]:
rblock_new(D)
rpar_new(D)
rgdump_griddata(dir)
rdump_griddata(dir,D)
misc_calc(calc_bu=1, calc_bsq=1)
bd=np.zeros_like(bu)
ud=np.zeros_like(uu)
bs1=bs1new
bs2=bs2new
bs3=bs3new
offset=0
print('r.max()',r.max())
bd[:,0] = mdot(gcov[:,:,0],bu[:,0])
ud[:,0] = mdot(gcov[:,:,0],uu[:,0])
rho[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]=0.5*(rho[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]+rho[:,:,bs2-3:bs2,offset])
rho[:,:,bs2-3:bs2,offset]=rho[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]
rho[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]=0.5*(rho[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]+rho[:,:,0:3,offset])
rho[:,:,0:3,offset]=rho[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]
ug[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]=0.5*(ug[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]+ug[:,:,bs2-3:bs2,offset])
ug[:,:,bs2-3:bs2,offset]=ug[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]
ug[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]=0.5*(ug[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]+ug[:,:,0:3,offset])
ug[:,:,0:3,offset]=ug[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]
bsq[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]=0.5*(bsq[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]+bsq[:,:,bs2-3:bs2,offset])
bsq[:,:,bs2-3:bs2,offset]=bsq[:,:,bs2-3:bs2,int(len(r[0,0,0,:])*.5)+offset]
bsq[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]=0.5*(bsq[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]+bsq[:,:,0:3,offset])
bsq[:,:,0:3,offset]=bsq[:,:,0:3,int(len(r[0,0,0,:])*.5)+offset]
_dx3=2.*np.pi/bs3
_dx2=np.pi/bs2
for ito in range(0,bs3):
    ph[:,:,:,ito]=_dx3/2.+np.float(ito)*_dx3
offset=0
for ito in range(0,bs2):
    h[:,:,ito,:]=_dx2/2.+np.float(ito)*_dx2
ph[:,:,:,0]=0.01
ph[:,:,:,-1]=2.*np.pi-0.01
h[:,:,0,:]=0.01
h[:,:,-1,:]=np.pi-0.01
_dx3=2.*np.pi/bs3new
#for grid file
dump_RT_KC(dir_BHOSS+“grid_HAMR3D_sgrA_weak_field_BALL.bin”,1,150)
#for grmhd->grrt file
dump_RT_KC(dir_BHOSS+“GRMHD_HAMR3D_sgrA_weak_field_%d.bin” %D,0,150)